In [1]:
import numpy as np
import pandas as pd
import regex as re
import os
from py_vncorenlp import VnCoreNLP
from transformers import pipeline

cwd = os.getcwd()
vncorenlp = VnCoreNLP(save_dir=os.environ["VNCORENLP"])
os.chdir(cwd)

2024-06-24 21:51:20 INFO  WordSegmenter:24 - Loading Word Segmentation model
2024-06-24 21:51:20 INFO  PosTagger:23 - Loading POS Tagging model
2024-06-24 21:51:22 INFO  NerRecognizer:34 - Loading NER model
2024-06-24 21:51:30 INFO  DependencyParser:32 - Loading Dependency Parsing model


In [2]:
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"


def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic


dicchar = loaddicchar()


def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt
    )

In [3]:
bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

In [4]:
sp_word_sub = {
    "@@": "confuseeyes",
    "℅": "%",
    r"/": " fraction ",
    r":\)+": "smileface",
    r";\)+": "smileface",
    r":\*+": "kissingface",
    r"=\)+": "playfulsmileface",
    r"=\(+": "playfulsadface",
    r":\(+": "sadface",
    r":3+": "threeface",
    r":v+": "vface",
    r"\^\^": "kindsmile",
    r"\^_\^": "kindmountsmile",
    r"\^\.\^": "kindmountsmile",
    r"-_-": "disapointface",
    r"\._\.": "confusedface",
    r":>+": "cutesmile",
    r"(\|)w(\|)": "fancycryface",
    r":\|": "mutedface",
    r":d+": "laughface",
    r"<3": "loveicon",
    r"\.{2,}": "threedot",
    r"-{1,}>{1,}": "arrow",
    r"={1,}>{1,}": "arrow",
    r"(\d+)h": r"\1 giờ",
    r"(\d+)'": r"\1 phút",
    r"(\d+)trieu": r"\1 triệu",
    r"(\d+)\s?tr": r"\1 triệu",
    r"blut\w+": "bluetooth",
    r"(\d+)\s\*": r"\1 sao"
}

replace_dict = {
    "/": "fraction",
    "wf": "wifi",
    "wifj": "wifi",
    "wjfj": "wifi",
    "wjfi": "wifi",
    "wiffi": "wifi",
    "wj": "wifi",
    "ko": "không",
    "k": "không",
    "hong": "không",
    "đc": "được",
    "sp": "sản phẩm",
    "fb": "facebook",
    "ytb": "youtube",
    "yt": "youtube",
    "mes": "messenger",
    "mess": "messenger",
    "tgdđ": "thegioididong",
    "nv": "nhân viên",
    "ss": "samsung",
    "ip": "iphone",
    "appel": "apple",
    "oke": "ok",
    "okie": "ok",
    "okey": "ok",
    "oki": "ok",
    "oce": "ok",
    "okela": "ok",
    "mk": "mình",
    "sd": "sử dụng",
    "sdung": "sử dụng",
    "ae": "anh em",
    "lq": "liên quân",
    "lqmb": "liên quân mobile",
    "lun": "luôn",
    "ng": "người",
    "ad": "admin",
    "ms": "mới",
    "cx": "cũng",
    "cũg": "cũng",
    "nhìu": "nhiều",
    "bth": "bình thường",
    "bthg": "bình thường",
    "ngta": "người ta",
    "dow": "download",
    "hdh": "hệ điều hành",
    "hđh": "hệ điều hành",
    "cammera": "camera",
    "dt": "điện thoại",
    "dthoai": "điện thoại",
    "dth": "điện thoại",
    "đth": "điện thoại",
    "hk": "không",
    "j": "gì",
    "ji": "gì",
    "mn": "mọi người",
    "m.n": "mọi người",
    "mjh": "mình",
    "mjk": "mình",
    "lắc": "lag",
    "lác": "lag",
    "lang": "lag",
    "nhah": "nhanh",
    "nóichung": "nói chung",
    "zl": "zalo",
    "sóg": "sóng",
    "rẽ": "rẻ",
    "trc": "trước",
    "chíp": "chip",
    "bin": "pin",
    "lm": "làm",
    "bik": "biết",
    "hog": "không",
    "zỏm": "dổm",
    "z": "vậy",
    "v": "vậy",
    "nhah": "nhanh",
    "r": "rồi",
    "ỗn": "ổn",
    "nhìu": "nhiều",
    "wá": "quá",
    "wep": "web",
    "wed": "web",
    "fim": "phim",
    "film": "phim",
    "xạc": "sạc",
    "xài": "sài",
    "het": "hết",
    "lun": "luôn",
    "e": "em",
    "a": "anh",
    "bjo": "bây giờ",
    "vl": "vãi lồn",
    "sac": "sạc",
    "vidieo": "video",
    "tét": "test",
    "tes": "test",
    "thik": "thích",
    "fai": "phải",
    "✋": "tay",
    "🔋": "pin",
    "☆": "sao",
    "supper": "super",
    "lổi": "lỗi",
    "loát": "load",
    "thui": "thôi",
    "rùi": "rồi",
    "ỗn": "ổn",
    "lổi": "lỗi",
    "suống": "xuống",
    "selfi": "selfie",
    "gg": "google",
    "cam on": "cảm ơn",
    "tg": "thời gian",
    "nchung": "nói chung",
    "❤": "loveicon",
    "trại nghiệm": "trải nghiệm",
    "dất": "rất",
    "đứg": "đứng",
    "bằg": "bằng",
    "mìh": "mình",
    "đag": "đang",
    "thoi": "thôi",
    "củng": "cũng",
    "đả": "đã",
    "màng": "màn",
    "ff": "free fire",
    "cod": "call of duty",
    "moi thứ": "mọi thứ",
    "moi thu": "mọi thứ",
    "moi thư": "mọi thứ",
    "moi người": "mọi người",
    "moi": "mới",
    "dk": "được",
    "đk": "được",
    "nhậy": "nhạy",
    "ak": "á",
    "ghe": "nghe",
    "bùn": "buồn",
    "bit": "biết",
    "bít": "biết",
    "bnhieu": "bao nhiêu",
    "dụg": "dụng",
    "tk": "tài khoản",
    "sąc": "sạc",
    "rât": "rât",
    "haz": "haiz",
    "sai làm": "sai lầm",
    "flim": "phim",
    "xướt": "xước",
    "viềng": "viền"
}

drop_indices = [1537, 2754, 4267, 6536]

def normalize(text: str, track_change=False):
    # Lowercase
    text = text.lower()

    text = re.sub(r"((https?|ftp|file):\/{2,3})+([-\w+&@#/%=~|$?!:,.]*)|(www.)+([-\w+&@#/%=~|$?!:,.]*)", "urllink", text)

    # Remove dup trailing chars (troiiiii -> troi)
    text = re.sub(r"([\D\w])\1+\b", r"\1", text)
    if track_change:
        print("Dedup trailing: ", text)

    # Replace special symbol to word
    for pttn, repl in sp_word_sub.items():
        text = re.sub(fr"{pttn}", f" {repl} ", text)
    if track_change:
        print("Replace special word: ", text)
    
    # Correct misspelled word
    def replace(match):
        orig = match.group(1)
        word = " " + replace_dict.get(orig, orig) + " "
        return word
    text = re.sub(r"\b(\S+)\b", replace, text)
    if track_change:
        print("Correct misspelled word: ", text)

    # Normalize string encoding
    text = convert_unicode(text)
    if track_change:
        print("Normalize string encoding: ", text)

    # Vietnamese unicode normalization
    text = chuan_hoa_dau_cau_tieng_viet(text)
    if track_change:
        print("Vietnamese unicode normalization: ", text)

    # Eliminate decimal delimiter (9.000 -> 9000)
    text = re.sub(r"(?<=\d)\.(?=\d{3})", "", text)
    if track_change:
        print("Eliminate decimal delimiter: ", text)
    
    # Split between value and unit (300km -> 300 km)
    text = re.sub(r"(\d+)(\D+)", r"\1 \2", text)
    if track_change:
        print("Split between value and unit: ", text)

    # Split by punctuations
    text = " ".join(
        re.split("(["+re.escape("!\"#$%&\'()*+,-./:;<=>?@[\\]^`{|}~")+"])", text)
    )
    if track_change:
        print("Split by punctuations: ", text)

    # Split by emoticons
    text = " ".join(
        re.split("(["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U0001f926-\U0001f937"
            u'\U00010000-\U0010ffff'
            u"\u200d"
            u"\u2640-\u2642"
            u"\u2600-\u2B55"
            u"\u23cf"
            u"\u23e9"
            u"\u231a"
            u"\u3030"
            u"\ufe0f"
            u"\u221a"
        "])", text)
    )

    # Word segmentation
    # text = " ".join(vncorenlp.word_segment(text))
    
    return text

In [5]:
normalize("Ủa cho em hỏi là iphone 7plus chính hãng của tgdd là không có loại 64G hay 128G dạ ? Hay là nó độc quyền 32G à . Còn mấy cái 7plus G cao là hàng nhái hay sao ? Tại em ra tgdd kiếm mà chỉ có 32G thôi nên mua luôn 😂 giờ thắc mắc lên hỏi 🤔🤔", track_change=True)

Dedup trailing:  ủa cho em hỏi là iphone 7plus chính hãng của tgd là không có loại 64g hay 128g dạ ? hay là nó độc quyền 32g à . còn mấy cái 7plus g cao là hàng nhái hay sao ? tại em ra tgd kiếm mà chỉ có 32g thôi nên mua luôn 😂 giờ thắc mắc lên hỏi 🤔🤔
Replace special word:  ủa cho em hỏi là iphone 7plus chính hãng của tgd là không có loại 64g hay 128g dạ ? hay là nó độc quyền 32g à . còn mấy cái 7plus g cao là hàng nhái hay sao ? tại em ra tgd kiếm mà chỉ có 32g thôi nên mua luôn 😂 giờ thắc mắc lên hỏi 🤔🤔
Correct misspelled word:   ủa   cho   em   hỏi   là   iphone   7plus   chính   hãng   của   tgd   là   không   có   loại   64g   hay   128g   dạ  ?  hay   là   nó   độc   quyền   32g   à  .  còn   mấy   cái   7plus   g   cao   là   hàng   nhái   hay   sao  ?  tại   em   ra   tgd   kiếm   mà   chỉ   có   32g   thôi   nên   mua   luôn  😂  giờ   thắc   mắc   lên   hỏi  🤔🤔
Normalize string encoding:   ủa   cho   em   hỏi   là   iphone   7plus   chính   hãng   của   tgd   là   không   có 

'ủa cho em hỏi là iphone 7 plus chính hãng của tgd là không có loại 64 g hay 128 g dạ  ?  hay là nó độc quyền 32 g à  .  còn mấy cái 7 plus g cao là hàng nhái hay sao  ?  tại em ra tgd kiếm mà chỉ có 32 g thôi nên mua luôn  😂  giờ thắc mắc lên hỏi  🤔  🤔 '

In [6]:
from tqdm import tqdm

for dset in ["Train", "Dev", "Test"]:
    df = pd.read_csv(f"data/{dset}.csv")
    if dset == "Train":
        df.drop(index=drop_indices)
    normalized = df.comment.map(normalize)
    with open(f"data/prep-{dset}.txt", "w") as f:
        f.write("\n".join(normalized))